# End to End XGBoost

©2022 MetaSnake

`@__mharrison__`

In [ ]:
# v1.4 please
!pip install -U yellowbrick

## Libraries
We will also use SHAP, xgbfir, openpyxl, hyperopt

In [ ]:
# for colab
!pip install dtreeviz feature_engine pybaobabdt xgbfir shap

In [ ]:
try:
    import dtreeviz
except ImportError:
    print("No dtreeviz")
from feature_engine import encoding, imputation
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
try:
    import pybaobabdt
except ImportError:
    print("No pybaobabdt")

from sklearn import base, compose, datasets, ensemble, \
    metrics, model_selection, pipeline, preprocessing, tree
import xgboost as xgb
import yellowbrick.model_selection as ms
from yellowbrick import classifier

import urllib
import zipfile

## Datasets

I'll be demoing with Titanic 🤔😉
Your lab will be with Kaggle data

In [ ]:
titanic_X, titanic_y = datasets.fetch_openml("titanic", version=1, as_frame=True, return_X_y=True)
titanic_y = titanic_y.astype(int)

In [ ]:
titanic_y

In [ ]:
class TweakTransformer(base.BaseEstimator, base.TransformerMixin):
    def __init__(self, ycol=None):
        self.ycol = ycol
        self.y_val = None
        
    def transform(self, X):
        df = (X
             .drop(columns=['name', 'ticket', 'home.dest', 'boat', 'body', 'cabin'])
            )
        if self.ycol in df:
            self.y_val = df[self.ycol]
        return df
    def fit(self, X, y=None):
        return self
        

titanic_pl = pipeline.Pipeline([('tweak', TweakTransformer()),
    ('cat_impute', imputation.CategoricalImputer(imputation_method='missing',
            variables=['sex', 'embarked'])),
    ('cat', encoding.OneHotEncoder(top_categories=5, drop_last=True, 
            variables=['sex', 'embarked'])),
    ('num_impute', imputation.MeanMedianImputer(imputation_method='median',
                                               variables=['age', 'fare']))
                       ])

titanic_X_train, titanic_X_test, titanic_y_train, titanic_y_test = model_selection.train_test_split(
    titanic_X, titanic_y, random_state=42, stratify=titanic_y)
titanic_pl.fit_transform(titanic_X_train, titanic_y_train)

In [ ]:
%ls -l

In [ ]:
url = 'https://github.com/mattharrison/datasets/raw/master/data/kaggle-survey-2018.zip'
fin = urllib.request.urlopen(url)
#fin = open('kaggle-survey-2018.zip', mode='rb')
data = fin.read()
with open('kaggle-survey-2018.zip', mode='wb') as fout:
    fout.write(data)
with zipfile.ZipFile('kaggle-survey-2018.zip') as z:
    print(z.namelist())
    kag = pd.read_csv(z.open('multipleChoiceResponses.csv'))
    kag_questions = kag.iloc[0]
    raw = kag.iloc[1:]

In [ ]:
def topn(ser, n=5, default='other'):
    counts = ser.value_counts()
    return ser.where(ser.isin(counts.index[:n]), default)

def tweak_kag(df):
    return (df
        #.query('Q3.isin(["United States of America", "China", "India"]) '\
        #       'and Q6.isin(["Data Scientist", "Software Engineer"])')
        .loc[df.Q3.isin(["United States of America", "China", "India"]) &
             df.Q6.isin(["Data Scientist", "Software Engineer"])]
        .pipe(lambda df_:
            df_.assign(**(df_.Q1.pipe(pd.get_dummies, drop_first=True, prefix='gender')),
                       age=df_.Q2.str.slice(0,2).astype(int),
                       **(df_.Q3.pipe(pd.get_dummies, drop_first=True, prefix='country')),
                       education=df_.Q4.replace({'Master’s degree': 18,
                         'Bachelor’s degree': 16,
                         'Doctoral degree': 20,
                         'Some college/university study without earning a bachelor’s degree': 13,
                         'Professional degree': 19,
                         'I prefer not to answer': None,
                         'No formal education past high school': 12}),
                       **(df_.Q5
                              .pipe(topn, n=3)
                              .replace({
                        'Computer science (software engineering, etc.)': 'cs',
                        'Engineering (non-computer focused)': 'eng',
                        'Mathematics or statistics': 'stat'})
                              .pipe(pd.get_dummies, drop_first=True, prefix='major')),
                       title=df_.Q6,
                       years_exp=(df_.Q8.str.replace('+','', regex=False)
                           .str.split('-', expand=True)
                           .iloc[:,0]
                           .astype(float)),
                       compensation=(df_.Q9.str.replace('+','', regex=False)
                           .str.replace(',','', regex=False)
                           .str.replace('500000', '500', regex=False)
                           .str.replace('I do not wish to disclose my approximate yearly compensation', '0', regex=False)
                           .str.split('-', expand=True)
                           .iloc[:,0]
                           .fillna(0)
                           .astype(int)
                           .mul(1_000)
                                    ),
                       python=df_.Q16_Part_1.fillna(0).replace('Python', 1),
                       r=df_.Q16_Part_2.fillna(0).replace('R', 1),
                       sql=df_.Q16_Part_3.fillna(0).replace('SQL', 1)
               )#assign
              
        )#pipe
        .rename(columns=lambda col:col.replace(' ', '_'))
        .loc[:, 'gender_Male':]   
        .dropna()
       )
kag = tweak_kag(raw)
kag_X = kag.drop(columns='title')
kag_y = (kag.title == 'Data Scientist')


In [ ]:
kag_y

## Stumps, Trees, and Forests

Decision trees use a greedy algorithm to split on a feature (column) that results in the most "pure" split.

In [ ]:
stump = tree.DecisionTreeClassifier(max_depth=1)
X_train = titanic_pl.fit_transform(titanic_X_train)
X_test = titanic_pl.transform(titanic_X_test)
stump.fit(X_train, titanic_y_train.astype(int))
stump.score(X_test, titanic_y_test.astype(int))

In [ ]:
# 0 - died, 1 - survived
stump.classes_

In [ ]:
# broken but fingers crossed for future
features = list(c for c in X_train.columns)
#plt.rcParams["font.family"] = "Roboto"
pybaobabdt.drawTree(stump, size=10, dpi=72, features=features)

In [ ]:
features = list(c for c in X_train.columns)
tree.plot_tree(stump, feature_names=features, filled=True, class_names=['Died', 'Survived'])

In [ ]:
stump.score(X_test, titanic_y_test.astype(int))

## Underfit
A stump is too simple. It has too much *bias*.

Solutions:

* Add more features
* Use a more complex model

For a tree we can let it grow deeper which should do both.

## Overfitting

A model is too complicated. It has too much variance.

Solutions:

* Simplify or constrain (*regularize*)
* Add more samples

For a tree we can prune back the growth so that the leaf nodes are overly specific.

In [ ]:
hi_variance = tree.DecisionTreeClassifier(max_depth=None)
X_train = titanic_pl.fit_transform(titanic_X_train)
hi_variance.fit(X_train, titanic_y_train)
hi_variance.score(X_test, titanic_y_test)

In [ ]:
titanic_y_train

In [ ]:
features = list(c for c in X_train.columns)
_=tree.plot_tree(hi_variance, feature_names=features, filled=True, class_names=['Died', 'Survived'])

In [ ]:
# limit view to first 2
features = list(c for c in X_train.columns)
_=tree.plot_tree(hi_variance, feature_names=features, filled=True, 
                 class_names=['Died', 'Survived'],
                max_depth=2)


## Tree Hyperparameters

*max_\** parameters - Raise to make more complex (overfit|more variance), lower to simplify (underfit|more bias)

*min_\** parameters - Lower to make more complex (overfit|more variance), raise to simplify (underfit|more bias)

* 'max_depth=None' - Tree depth
* 'max_features=None' - Amount of features to examine for split
* 'max_leaf_nodes=None' - Number of leafs
* 'min_impurity_decrease=0' - Split when *impurity* is >= this value. (*Impurity* : 0 - 100% accurate, .3 - 70%. Going from 70% to 100% accurate is a decrease of .3) 
* 'min_samples_leaf=1', - Minimum samples at each leaf.
* 'min_samples_split=2' - Minimum samples required to split a node.
* 'min_weight_fraction_leaf=0' - The fraction fo the total weights required to be a leaf.


In [ ]:
print(dir(stump))

## Random Forest

Uses *bagging* to ensemble many trees in an attempt to lower variance.

In [ ]:
rf = ensemble.RandomForestClassifier(random_state=42)
rf.fit(X_train, titanic_y_train)
rf.score(X_test, titanic_y_test)

In [ ]:
len(rf.estimators_)

In [ ]:
features = list(c for c in X_train.columns)
_=tree.plot_tree(rf.estimators_[0], feature_names=features, filled=True, class_names=['Died', 'Survived'],
                max_depth=2)

## Random Forest Hyperparameters

*max_\** parameters - Raise to make more complex (overfit|more variance), lower to simplify (underfit|more bias)

*min_\** parameters - Lower to make more complex (overfit|more variance), raise to simplify (underfit|more bias)

* 'n_estimators=100' - Number of trees - should be *max_estimators*
* 'oob_score=False' - Can estimate score when training (by using rows that weren't randomly selected). No need to hold out data
* 'warm_start=False' - Can add more trees w/o starting over

From tree:

* 'max_depth=None' - Tree depth (1 to Infinity (`None`))
* 'max_features="sqrt"' - Amount of features to examine for split (1 to number of features (int). Float of percent (0. to 1.0). "log2" log2(n_features) or "sqrt"  sqrt(n_features). (Default square root number of features.)
* 'max_leaf_nodes=None' - Number of leafs. Default (`None`) is unlimited.
* 'min_impurity_decrease=0' - Split when *impurity* is >= this value. (0.0 to 1.0) (*Impurity* : 0 - 100% accurate, .3 - 70%) 
* 'min_samples_leaf=1', - Minimum samples at each leaf. (1 to n_samples).
* 'min_samples_split=2' - Minimum samples required to split a node. (1 to n_samples)
* 'min_weight_fraction_leaf=0' - The fraction (0.0 to 1.0) of the total weights required to be a leaf.

In [ ]:
print(dir(rf))

In [ ]:
# Show score estimated during training
rf_oob = ensemble.RandomForestClassifier(random_state=42, oob_score=True)
rf_oob.fit(X_train, titanic_y_train)
rf_oob.score(X_test, titanic_y_test), rf_oob.oob_score_

In [ ]:
# Notice that the .score result changes after raising n_estimators
rf_ws = ensemble.RandomForestClassifier(random_state=42, oob_score=True, 
                                        warm_start=True, n_estimators=3)
rf_ws.fit(X_train, titanic_y_train)
print(rf_ws.score(X_test, titanic_y_test), rf_oob.oob_score_)

# change parameter and call .fit again. Doesn't need to start over
rf_ws.set_params(n_estimators=200)
rf_ws.fit(X_train, titanic_y_train)
rf_ws.score(X_test, titanic_y_test), rf_oob.oob_score_

In [ ]:
# visualize how changing n_estimators affects score
results = []
rf_ws = ensemble.RandomForestClassifier(random_state=42, warm_start=True, n_estimators=1)
rf_ws.fit(X_train, titanic_y_train)
for i in range(2,100):
    rf_ws.set_params(n_estimators=i)
    rf_ws.fit(X_train, titanic_y_train)
    results.append(rf_ws.score(X_test, titanic_y_test))
    #results.append(metrics.f1_score(titanic_y_test, rf_ws.predict(X_test)))
    #results.append(metrics.precision_score(titanic_y_test, rf_ws.predict(X_test)))
    #results.append(metrics.recall_score(titanic_y_test, rf_ws.predict(X_test)))
    #results.append(metrics.roc_auc_score(titanic_y_test, rf_ws.predict(X_test)))
pd.Series(results).plot(figsize=(8,4))    

## Tree Exercise

* Create a decision tree for the Kaggle Data predicting whether title is data scientist or software engineer
* What is the score?
* Create a random forest for the Kaggle Data predicting whether title is data scientist or software engineer
* What is the score?

## XGBoost

Uses *boosting* to train a series of (weak) trees that try to correct the error of the previous output. (For classification this is mapped to a probability)

Like golfing (you continue to putt or use a different club depending on first error). Decision tree would be a single tee off. Random forest would be averaging the tee offs. 

* Regularization
* Parallel Processing
* Missing Number Support
* Category Support

In [ ]:
xg = xgb.XGBClassifier(eval_metric='logloss', use_label_encoder=False)
xg.fit(X_train, titanic_y_train.astype(int))
xg.score(X_test, titanic_y_test.astype(int))

In [ ]:
# Let's try w/ depth of 2 and 2 trees
xg = xgb.XGBClassifier(eval_metric='logloss', use_label_encoder=False, max_depth=2, n_estimators=2)
xg.fit(X_train, titanic_y_train.astype(int))
xg.score(X_test, titanic_y_test.astype(int))

In [ ]:
# first tree
xgb.to_graphviz(xg, size='1,1', num_trees=0, fontsize='1')

In [ ]:
# second tree
xgb.to_graphviz(xg, size='1,1', num_trees=1, fontsize='1')

In [ ]:
# let's go down the left path with
# this data
row = pd.Series({'pclass': 2.0,
 'age': 28.0,
 'sibsp': 8.0,
 'parch': 2.0,
 'fare': 69.55,
 'sex_male': 0.0,
 'sex_female': 1.0,
 'sex_Missing': 0.0,
 'embarked_S': 1.0,
 'embarked_C': 0.0,
 'embarked_Q': 0.0,
 'embarked_Missing': 0.0}).to_frame().T

row

In [ ]:
# result for survival = .7027
# > .5 ... so Survives!
# this is [prob death, prob survival]
xg.predict_proba(row)

In [ ]:
xg.predict(row)

In [ ]:
# sum up leafs and throw into 
# Example: male, class 3
# .49 + .37

vals = np.linspace(-10, 10)
def inv_logit(p):
    return np.exp(p) / (1 + np.exp(p))

print(inv_logit(.49+.37))
fig, ax = plt.subplots(figsize=(6,4))
ax.plot(vals, inv_logit(vals))

## XGBoost Exercise

* Create an XGBoost tree for the Kaggle Data predicting whether title is data scientist or software engineer
* What is the score?


## Early Stopping
Because you can keep "putting" you can keep track of how far away you are from the hole and stop when you are closest.

In [ ]:
# defaults
# 100 putts
xg = xgb.XGBClassifier(eval_metric='logloss', use_label_encoder=False)
xg.fit(X_train, titanic_y_train.astype(int))
xg.score(X_test, titanic_y_test.astype(int))

In [ ]:
# Early stopping
# Go up to 100 but stop after you haven't improved for 20 hits
# Min value at round 10 validation_1-logloss:0.40012

xg = xgb.XGBClassifier(eval_metric='logloss', use_label_encoder=False)
xg.fit(X_train, titanic_y_train.astype(int),
       eval_set=[(X_train, titanic_y_train.astype(int)),
                 (X_test, titanic_y_test.astype(int))],
            early_stopping_rounds=20) 
xg.score(X_test, titanic_y_test.astype(int))


In [ ]:
xg.best_ntree_limit

In [ ]:
# we can get the evaluation metrics
# validation_0 is for training data
# validation_1 is for testing data
results = xg.evals_result()
results

In [ ]:
# we can get the evaluation metrics
results['validation_0']['logloss']

In [ ]:
# Testing score is best at 11 trees
results = xg.evals_result()
ax = pd.DataFrame({'training': results['validation_0']['logloss'],
              'testing': results['validation_1']['logloss'],
             }).shift().plot(figsize=(5,4))
ax.set_xlabel('ntrees')

## XGBoost Hyperparameters

*max_\** parameters - Raise to make more complex (overfit|more variance), lower to simplify (underfit|more bias)

*min_\** parameters - Lower to make more complex (overfit|more variance), raise to simplify (underfit|more bias)

* Boosting

  * ``n_estimators=100`` - number of trees (or boosting rounds). Larger is more complex. Default 100. Use ``early_stopping_rounds`` with ``.fit`` to prevent overfitting.

  * ``learning_rate=.3`` (called ``eta`` too) - after each boosting step, shrink feature weights. Larger is more conservative. Can be used with n_estimators to adjust time for convergence [0,1], default .3

  * ``gamma=0`` / ``min_split_loss`` - L0 regularization. Global regularization. Minimum loss required for split. Larger is more conservative. [0, ∞], default 0 - No regularization.


* Regularization

  * ``reg_lambda=1`` - L2 regularization (Root of squared weights). Increase to be more conservative. Default 1
  * ``reg_alpha=0`` - L1 regularization (Mean of weights). Increase to be more conservative. Default 0

* Sampling - Use different rows

  * ``subsample=1`` - Use % of samples (this is rows!) for next boosting round. Lower to more conservative. [0, 1], default 1. (When not equal to 1.0, model does *stochastic gradient descent*, ie. there is some randomness in the model.)


New tree (sampling) parameters - Use different columns (not rows!):

  * ``colsample_bytree=1`` - Fraction of columns for each boosting round.
  
  * ``colsample_bylevel=1`` - Fraction of columns for each depth level.
  
  * ``colsample_bynode=1`` - Fraction of columns for each node.
  

From tree:

  * ``max_depth=6`` - depth of tree. Larger is more complex (more likely to overfit). How many feature interactions you can have. Each level doubles time. [0, ∞], default 6
  * ``min_child_weight=1`` - Stop splitting after certain amount of purity. Larger will be more conservative.


Imbalanced data:

* ``scale_pos_weight=1`` -  ratio negative/positive. Default 1
* Use ``'auc'`` or ``'aucpr'`` for ``eval_metric`` metric (rather than classification default ``'logless'``)
* ``max_delta_step=0`` - try values from 1-10. Default 0





In [ ]:
fig, ax = plt.subplots(figsize=(8,4))
ms.validation_curve(xgb.XGBClassifier(eval_metric='logloss', use_label_encoder=False), X_train, titanic_y_train.astype(int),
                    param_name='gamma', param_range=[0, .5, 1,2,5,10, 20])

In [ ]:
fig, ax = plt.subplots(figsize=(8,4))
ms.validation_curve(tree.DecisionTreeClassifier(), X_train, titanic_y_train.astype(int),
                    param_name='max_depth', param_range=[1,2,5,10])

In [ ]:
fig, ax = plt.subplots(figsize=(8,4))
ms.validation_curve(xgb.XGBClassifier(eval_metric='logloss', use_label_encoder=False), X_train, titanic_y_train.astype(int),
                    param_name='max_depth', param_range=[1,2,5,10])

In [ ]:
# note this depends on n_estimators
# should really use early stopping but yellowbrick doesn't support this 😢
fig, ax = plt.subplots(figsize=(8,4))
ms.validation_curve(xgb.XGBClassifier(eval_metric='logloss', use_label_encoder=False), X_train, titanic_y_train.astype(int),
                    param_name='learning_rate', param_range=[0.001, .01, .1, .2, .5, .9, 1])

In [ ]:
# this takes a while to run (about 2 minutes)
# can set scoring in GridSearchCV to 
# recall, precision, f1, accuracy
params = {'reg_lambda': [0],  # No effect
          'learning_rate': [.1, .3], # makes each boost more conservative (0 - no shrinkage) 
          'subsample': [.7, 1],
          #'gamma': [0, 1],
          'max_depth': [2, 3],
          'random_state': [42],
          'n_jobs': [-1],
          'n_estimators': [200]}
xgb2 = xgb.XGBClassifier(eval_metric='logloss', use_label_encoder=False)
cv = (model_selection.GridSearchCV(xgb2, params, cv=3, n_jobs=-1)
    .fit(X_train, titanic_y_train.astype(int),
         eval_set=[(X_test, titanic_y_test.astype(int))],
         early_stopping_rounds=5) 
     )

In [ ]:
cv.best_params_

In [ ]:
params = {'learning_rate': 0.1,
 'max_depth': 3,
 'n_estimators': 200,
 'n_jobs': -1,
 'random_state': 42,
 'reg_lambda': 0,
 'subsample': 1}

In [ ]:
# vs default
xgb_def = xgb.XGBClassifier(eval_metric='logloss', use_label_encoder=False)
xgb_def.fit(X_train, titanic_y_train.astype(int))
#xgb_grid = xgb.XGBClassifier(**cv.best_params_, eval_metric='logloss', use_label_encoder=False)
xgb_grid = xgb.XGBClassifier(**params, eval_metric='logloss', use_label_encoder=False)
xgb_grid.fit(X_train, titanic_y_train.astype(int))
xgb_def.score(X_test, titanic_y_test.astype(int)), xgb_grid.score(X_test, titanic_y_test.astype(int))

## Hyperparameter Exercise

* Use grid search to evaluate hyperparameters for your model

## Bonus: Tuning with Hyperopt


In [ ]:
!pip install hyperopt

In [ ]:
# our basic grid search example
params = {'learning_rate': 0.1,
 'max_depth': 3,
 'n_estimators': 200,
 'n_jobs': -1,
 'random_state': 42,
 'reg_lambda': 0,
 'subsample': 1}

In [ ]:
X_train = titanic_pl.fit_transform(titanic_X_train)
X_test = titanic_pl.transform(titanic_X_test)

In [ ]:
# vs default
xgb_def = xgb.XGBClassifier(eval_metric='logloss', use_label_encoder=False)
xgb_def.fit(X_train, titanic_y_train.astype(int))
xgb_grid = xgb.XGBClassifier(**params, eval_metric='logloss', use_label_encoder=False)
xgb_grid.fit(X_train, titanic_y_train.astype(int))
xgb_def.score(X_test, titanic_y_test.astype(int)), xgb_grid.score(X_test, titanic_y_test.astype(int))

In [ ]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import accuracy_score  
#https://bradleyboehmke.github.io/xgboost_databricks_tuning/index.html#slide21
space = {
    'learning_rate': hp.loguniform('learning_rate', -7, 0),
    'max_depth': hp.quniform('max_depth', 1, 12, 1),
    'min_child_weight': hp.loguniform('min_child_weight', -2, 3),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
    'gamma': hp.loguniform('gamma', -10, 10),
    'reg_alpha': hp.loguniform('alpha', -10, 10),
    'reg_lambda': hp.loguniform('lambda', -10, 10),
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'seed': 123,
}

In [ ]:
def hyperparameter_tuning(space):    
    model = xgb.XGBClassifier(max_depth = int(space['max_depth']), 
                gamma = space['gamma'],                                         
                reg_alpha = int(space['reg_alpha']),
                min_child_weight=space['min_child_weight'],                                 
                colsample_bytree=space['colsample_bytree'])
    evaluation = [(X_train, titanic_y_train.astype(int)),
            (X_test, titanic_y_test.astype(int))]
    model.fit(X_train, titanic_y_train.astype(int),
                 eval_set=evaluation, eval_metric="rmse",            
                 early_stopping_rounds=10,verbose=False)    
         
    pred = model.predict(X_test)
    accuracy = accuracy_score(titanic_y_test.astype(int), pred>0.5)    
    print ("SCORE:", accuracy)    
    #change the metric if you like    
    return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [ ]:
trials = Trials()
best = fmin(fn=hyperparameter_tuning,            
    space=space,           
    algo=tpe.suggest,            
    max_evals=1000,            
    trials=trials,
    #timeout=60*5 # 5 minutes
           )
print (best)

In [ ]:
best # new

In [ ]:
hyper_params ={'alpha': 0.0035215052171286097,
 'colsample_bytree': 0.5362587947646607,
 'gamma': 0.000561052978838129,
 'lambda': 0.037563373793385646,
 'learning_rate': 0.002795623033880572,
 'max_depth': 3,
 'min_child_weight': 0.5031984001674933,
 'subsample': 0.9264026302532845}
xgb_hyp = xgb.XGBClassifier(**hyper_params, eval_metric='logloss', 
                            use_label_encoder=False,
                           n_estimators=2_000)
evaluation = [(X_train, titanic_y_train.astype(int)),
            (X_test, titanic_y_test.astype(int))]
xgb_hyp.fit(X_train, titanic_y_train.astype(int), early_stopping_rounds=10,
           eval_set=evaluation)
xgb_hyp.score(X_test, titanic_y_test.astype(int))#

In [ ]:
xgb_hyp.score(X_test, titanic_y_test.astype(int))#

In [ ]:
# vs default and grid
xgb_def.score(X_test, titanic_y_test.astype(int)), xgb_grid.score(X_test, titanic_y_test.astype(int))

## Model Evaluation
Now that we've tuned our model, let's look at how it performs

In [ ]:
hyper_params ={'alpha': 0.0035215052171286097,
 'colsample_bytree': 0.5362587947646607,
 'gamma': 0.000561052978838129,
 'lambda': 0.037563373793385646,
 'learning_rate': 0.002795623033880572,
 'max_depth': 3,
 'min_child_weight': 0.5031984001674933,
 'subsample': 0.9264026302532845}
xgb_hyp = xgb.XGBClassifier(**hyper_params, eval_metric='logloss', 
                            use_label_encoder=False,
                           n_estimators=2_000)
evaluation = [(X_train, titanic_y_train.astype(int)),
            (X_test, titanic_y_test.astype(int))]
xgb_hyp.fit(X_train, titanic_y_train.astype(int), early_stopping_rounds=10,
           eval_set=evaluation)
xgb_hyp.score(X_test, titanic_y_test.astype(int))#

In [ ]:
metrics.accuracy_score(titanic_y_test.astype(int), xgb_hyp.predict(X_test))

In [ ]:
fig, ax = plt.subplots(figsize=(8,4))
classifier.confusion_matrix(xgb_hyp, X_train, titanic_y_train.astype(int),
                            X_test, titanic_y_test.astype(int),
                            classes=['Died', 'Survived']
                           )

In [ ]:
fig, ax = plt.subplots(figsize=(8,4))
metrics.RocCurveDisplay.from_estimator(xgb_hyp,
                       X_test, titanic_y_test.astype(int),ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(8,4))
classifier.precision_recall_curve(xgb_hyp, X_train, titanic_y_train.astype(int),
                   X_test, titanic_y_test.astype(int),
                   classes=['Died', 'Survived'],
                   micro=False, macro=False
                   )

In [ ]:
fig, ax = plt.subplots(figsize=(8,4))
classifier.classification_report(xgb_hyp, X_train, titanic_y_train.astype(int),
                   X_test, titanic_y_test.astype(int),
                   classes=['Died', 'Survived'],
                   micro=False, macro=False
                   )

## Evaluation Exercise
* Create a Confusion Matrix for your model
* Create an ROCAUC curve 
* Create a PrecisionRecall curve
* How is the model performing?

## Training For Different Metrics

We tuned our model. But we tuned it against accuracy. What if we want to optimize for recall?

In [ ]:
# accuracy tuning
fig, ax = plt.subplots(figsize=(8,4))
ms.validation_curve(xgb.XGBClassifier(eval_metric='logloss', use_label_encoder=False), 
                    X_train, titanic_y_train.astype(int),
    #                param_name='max_depth', param_range=[1,2,5,10]
                    param_name='learning_rate', param_range=[0.001, .01, .1, .2, .5, .9, 1]
    )

In [ ]:
# recall tuning
fig, ax = plt.subplots(figsize=(8,4))
ms.validation_curve(xgb.XGBClassifier(eval_metric='logloss', use_label_encoder=False), 
                    X_train, titanic_y_train.astype(int),
                    scoring='recall',
                    #param_name='max_depth', param_range=[1,2,5,10]
                    param_name='learning_rate', param_range=[0.001, .01, .1, .2, .5, .9, 1]
                   )

In [ ]:
fig, ax = plt.subplots(figsize=(8,4))
ms.validation_curve(xgb.XGBClassifier(eval_metric='logloss', use_label_encoder=False), 
                    X_train, titanic_y_train.astype(int),
                    scoring='f1',
                    param_name='max_depth', param_range=[1,2,5,10]
#                    param_name='learning_rate', param_range=[0.001, .01, .1, .2, .5, .9, 1]
                   )

## Model Interpretation

In [ ]:
# Trees are great when they overfit... They can explain what they overfit
# (You can use these for "surrogate models")
hi_variance = tree.DecisionTreeClassifier(max_depth=None)
hi_variance.fit(X_train, titanic_y_train)
hi_variance.score(X_test, titanic_y_test)

In [ ]:
# Feature importance shows the magnitude (not direction) of impact
(pd.Series(hi_variance.feature_importances_, index=X_train.columns)
 .sort_values()
 .plot.barh()
)

In [ ]:
# Visualize what happens
dt3 = tree.DecisionTreeClassifier(max_depth=3)
dt3.fit(X_train, titanic_y_train.astype(int))

dtreeviz.trees.dtreeviz(dt3, X_train, titanic_y_train.astype(int), target_name='survivied',
                       feature_names=X_train.columns, class_names=['died', 'survived'])

In [ ]:
# XGBoost also supports feature importance
xgb_def = xgb.XGBClassifier(eval_metric='logloss', use_label_encoder=False)
xgb_def.fit(X_train, titanic_y_train.astype(int))

In [ ]:
(pd.Series(xgb_def.feature_importances_, index=X_train.columns)
 .sort_values()
 .plot.barh()
)

In [ ]:
# * "weight" is the number of times a feature appears in a tree
# * "gain" is the average gain of splits which use the feature
# * "cover" is the average coverage of splits which use the feature
xgb.plot_importance(xgb_def, importance_type='cover')

## Explaining the Model Exercise
* What are the important features?


## xgbfir (Feature Interactions Reshaped)
 *Gain*: Total gain of each feature or feature interaction
 
 *FScore*: Amount of possible splits taken on a feature or feature Interaction
 
 *wFScore*: Amount of possible splits taken on a feature or feature nteraction weighted by the probability of the splits to take place
 
 *Average wFScore*: wFScore divided by FScore
 
 *Average Gain*: Gain divided by FScore
 
 *Expected Gain*: Total gain of each feature or feature interaction weighted by the probability to gather the gain


In [ ]:
!pip install openpyxl

In [ ]:
import xgbfir
xgbfir.saveXgbFI(xgb_def, feature_names=X_train.columns, OutputXlsxFile='fir.xlsx')
pd.read_excel('fir.xlsx')

In [ ]:
pd.read_excel('fir.xlsx', sheet_name='Interaction Depth 1')

In [ ]:
pd.read_excel('fir.xlsx', sheet_name='Interaction Depth 2')

## Feature Interactions Exercise
* What are the important features?
* What are the second level interactions?
* What are the third level interactions?


# SHAP (SHapley Additive exPlantations)
Should be *globally* consistent and accurate

 Shapley value (SHAP).
 
 From game theory, indicates how to distribute attribution of label



In [ ]:
import shap
shap.initjs()

# make sure you initialize the js side
shap_ex = shap.TreeExplainer(xgb_def)
vals = shap_ex.shap_values(X_test)

In [ ]:
# Let's explain an individual
X_test.iloc[0]

In [ ]:
xgb_def.predict(X_test.iloc[[0]])  # predicts death... why?

In [ ]:
# label is also death
titanic_y_test.iloc[0]

In [ ]:
# values show direction of feature impact
# for this individual
pd.Series(vals[0], index=X_test.columns).plot.barh()

In [ ]:
# the base value. We sum up the scores.
# > 0 Positive Case
shap_ex.expected_value

In [ ]:
# < 0 therefore ... Death
shap_ex.expected_value + vals[0].sum()

In [ ]:
# use matplotlib if having js issues
# blue - Dead
# red - Survival
shap.force_plot(shap_ex.expected_value, 
               vals[0,:], X_test.iloc[0], #matplotlib=True
               )

In [ ]:
# Explain a feature
shap.dependence_plot('age', vals, X_test)

In [ ]:
# Explain another feature
shap.dependence_plot('fare', vals, X_test)

In [ ]:
# Explain a feature with an interaction
shap.dependence_plot('fare', vals, X_test, interaction_index='age')

In [ ]:
# Explain global features
shap.summary_plot(vals, X_test)

## Explaining the Model Exercise
* Use SHAP to explore predictions for a sample
* Use SHAP to expore a feature
* Use SHAP to explore the summary

## Other XGBoost Features

* Need to when predicting use the tree limit
* Monotonic constraints - Force relationship of column to target to be monotonic
* Interaction constraints - Limit interactions between features.
* Can simulate decision trees (just one estimator) and random forests.

## Tree Limit
Best practice is to use `early_stopping_rounds` with `eval_set` in call to `.fit`.

If you set `n_estimators` in the constructor (`xgb.XGBClassifier`) and set those, call to `.predict` and `.score` will only use `.best_ntree_limit` trees, not necessaritly `n_estimators`.

In [ ]:
hyper_params ={'alpha': 0.0035215052171286097,
 'colsample_bytree': 0.5362587947646607,
 'gamma': 0.000561052978838129,
 'lambda': 0.037563373793385646,
 'learning_rate': 0.002795623033880572,
 'max_depth': 3,
 'min_child_weight': 0.5031984001674933,
 'subsample': 0.9264026302532845}
xgb_hyp = xgb.XGBClassifier(**hyper_params, eval_metric='logloss', 
                            use_label_encoder=False,
                           n_estimators=2_000)
evaluation = [(X_train, titanic_y_train.astype(int)),
            (X_test, titanic_y_test.astype(int))]
xgb_hyp.fit(X_train, titanic_y_train.astype(int), early_stopping_rounds=10,
           eval_set=evaluation)
xgb_hyp.score(X_test, titanic_y_test.astype(int))#

In [ ]:
# Testing score is not best at 2_000 trees
results = xgb_hyp.evals_result()
ax = pd.DataFrame({'training': results['validation_0']['logloss'],
              'testing': results['validation_1']['logloss'],
             }).plot(figsize=(5,4))
ax.set_xlabel('ntrees')

In [ ]:
xgb_hyp#.booster.best_ntree_limit

In [ ]:
xgb_hyp.best_ntree_limit

In [ ]:
# uses best_ntree_limit
xgb_hyp.score(X_test, titanic_y_test.astype(int))

In [ ]:
metrics.accuracy_score(titanic_y_test.astype(int),
    xgb_hyp.predict(X_test, ntree_limit=xgb_hyp.best_ntree_limit)
)

In [ ]:
# using all of the trees is not necessarily better
metrics.accuracy_score(titanic_y_test.astype(int),
    xgb_hyp.predict(X_test, ntree_limit=1521)
)

## Monotonic Constraints
If you want to remove u or w shaped behavior in features, force a monotonic constraint

In [ ]:
# no constraints
hyper_params ={'alpha': 0.0035215052171286097,
 'colsample_bytree': 0.5362587947646607,
 'gamma': 0.000561052978838129,
 'lambda': 0.037563373793385646,
 'learning_rate': 0.002795623033880572,
 'max_depth': 3,
 'min_child_weight': 0.5031984001674933,
 'subsample': 0.9264026302532845}
xgb_hyp = xgb.XGBClassifier(**hyper_params, eval_metric='logloss', 
                            use_label_encoder=False,
                           n_estimators=2_000)
evaluation = [(X_train, titanic_y_train.astype(int)),
            (X_test, titanic_y_test.astype(int))]
xgb_hyp.fit(X_train, titanic_y_train.astype(int), early_stopping_rounds=10,
           eval_set=evaluation)
xgb_hyp.score(X_test, titanic_y_test.astype(int))#

In [ ]:
# add monotonic constraint to age
# as age goes up survival goes down!
hyper_params ={'alpha': 0.0035215052171286097,
 'colsample_bytree': 0.5362587947646607,
 'gamma': 0.000561052978838129,
 'lambda': 0.037563373793385646,
 'learning_rate': 0.002795623033880572,
 'max_depth': 3,
 'min_child_weight': 0.5031984001674933,
 'subsample': 0.9264026302532845}
# if age goes up survived (y) goes down (-1)
constraints = [0 if col != 'age' else -1 
              for col in X_train]
cst = f'({",".join(map(str,constraints))})'
# needs to be string '(1,0,0,-1,0,...)'
xgb_hyp_age = xgb.XGBClassifier(**hyper_params, eval_metric='logloss', 
                                monotone_constraints=cst,
                                use_label_encoder=False,
                               n_estimators=2_000)
evaluation = [(X_train, titanic_y_train.astype(int)),
            (X_test, titanic_y_test.astype(int))]
xgb_hyp_age.fit(X_train, titanic_y_train.astype(int), early_stopping_rounds=10,
           eval_set=evaluation)
xgb_hyp_age.score(X_test, titanic_y_test.astype(int))

In [ ]:
xgb_hyp_age.score(X_test, titanic_y_test.astype(int))

In [ ]:
# our score with constraints is worse on accuracy
xgb_hyp.score(X_test, titanic_y_test.astype(int))

In [ ]:
import shap
shap.initjs()

# make sure you initialize the js side
shap_ex = shap.TreeExplainer(xgb_hyp_age)
vals = shap_ex.shap_values(X_test)
shap.dependence_plot('age', vals, X_test)

In [ ]:
# no constrains
import shap
shap.initjs()

# make sure you initialize the js side
shap_ex = shap.TreeExplainer(xgb_hyp)
vals = shap_ex.shap_values(X_test)
shap.dependence_plot('age', vals, X_test)

## Interaction Constraints
You can limit what columns interact with other columns

In [ ]:
# looks like sex_male has an interaction with pclass (and age)
fig, ax = plt.subplots(figsize=(20,30), dpi=300)
xgb.plot_tree(xgb_hyp, rankdir='LR', num_trees=0, ax=ax)

In [ ]:
# need to specify index numbers of columns that can interact
print(list(enumerate(X_train.columns)))

In [ ]:
# make it so pclass and sex_male|sex_female can't interact
hyper_params ={'alpha': 0.0035215052171286097,
 'colsample_bytree': 0.5362587947646607,
 'gamma': 0.000561052978838129,
 'lambda': 0.037563373793385646,
 'learning_rate': 0.002795623033880572,
 'max_depth': 3,
 'min_child_weight': 0.5031984001674933,
 'subsample': 0.9264026302532845}
# Needs to be string with nested list of index values
interaction_constraints = str([[idx for idx, col in enumerate(X_train.columns)
                                if col not in {'pclass', 'sex_male', 'sex_female'}]])
# column names don't work (even though docs say they do)
#interaction_constraints = str([[col for idx, col in enumerate(X_train.columns)
#                                if col not in {'pclass', 'sex_male', 'sex_female'}]]).replace("'", '"')

xgb_hyp_age = xgb.XGBClassifier(**hyper_params, eval_metric='logloss', 
                                interaction_constraints=interaction_constraints,
                                use_label_encoder=False,
                               n_estimators=2_000)
evaluation = [(X_train, titanic_y_train.astype(int)),
            (X_test, titanic_y_test.astype(int))]
xgb_hyp_age.fit(X_train, titanic_y_train.astype(int), early_stopping_rounds=10,
           eval_set=evaluation)
xgb_hyp_age.score(X_test, titanic_y_test.astype(int))


In [ ]:
xgb_hyp_age.score(X_test, titanic_y_test.astype(int))

In [ ]:
fig, ax = plt.subplots(figsize=(20,30), dpi=300)
xgb.plot_tree(xgb_hyp_age, rankdir='LR', num_trees=0, ax=ax)

In [ ]:
import xgbfir
xgbfir.saveXgbFI(xgb_hyp_age, feature_names=X_train.columns, OutputXlsxFile='titanic_fir_interactions.xlsx')
pd.read_excel('titanic_fir_interactions.xlsx')

In [ ]:
# looks like pclass has a non-monotonic effect
pd.read_excel('titanic_fir_interactions.xlsx', sheet_name='Interaction Depth 1')

In [ ]:
pd.read_excel('titanic_fir_interactions.xlsx', sheet_name='Interaction Depth 2')

# Summary

XGBoost is very powerful. Combining with other tools will take you a long way.

Explore your data and your results.

Lots of libraries. Some are better integrated.

Suggestions:

* Pandas skills come in useful for manipulating data
* Make sure you discuss business value with stake holders


Questions?


Connect on LinkedIn or Twitter `@__mharrison__`